## 1. ADD Data sets

## 2. ASV- LA Data sets

In [1]:
import librosa
import datetime
import tensorflow as tf
import numpy as np
import os

from utils.DataLoader import data_loader
from utils.Generator0_blur import DataGenerator, feature_extract_cqt, evalEER,  evalScore, evalEER_f, evalEER_f2, gen_fname
from models.models import get_ResMax, get_LCNN, sigmoidal_decay
from models.models2 import get_BCResMax, get_DDWSseq
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import add,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, maximum, DepthwiseConv2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, softmax, swish
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import roc_curve

import pickle

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

add2022 = '/home/ubuntu/data/ADD/'
asv2019 = '/home/ubuntu/data/asv2019/'

pathset = { 'add2022' : add2022 , 'asv2019':asv2019}
        
dl = data_loader(pathset)

#dl.get_data(data_pick = '2', tde_pick = 't', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'd', pl_pick = 'l', to = 't')
#dl.get_data(data_pick = '2', tde_pick = 'e', pl_pick = 'l', to = 'd')


datapick = '2' ## 1:ADD, 2:LA

dl.get_data(data_pick = datapick, tde_pick = 't', pl_pick = 'l',  to = 't')
dl.get_data(data_pick = datapick, tde_pick = 'd', pl_pick = 'l', to = 'd')
dl.get_data(data_pick = datapick, tde_pick = 'e', pl_pick = 'l', to = 'e')

##################################################
mname = "ResMax_LA_"
##################################################

### Mixup
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_ResMax(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)
        
##################################################
mname = "DDWSseq_LA_"
##################################################

## 22.06.07 : 서버 점검으로 다시 돌리기(HP 2개까지 저장됨)
### HP         
for i in range(1) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

        
### LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = False #[0.5, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = False #[0.5, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5#np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = False #[2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)

### HP + LP + RP
for i in range(3) :
    sr = 16000
    sec = 9.0
    batch_size = 16
    feature = "cqt"
    #hop_length = 128
    #win_length = 512
    #n_bins = 80
    filter_scale = 1
    n_bins = 100
    fmin = 5
    epoch = 70
    beta_param = 0.7
    dropout_rate = 0.5 #np.random.choice([0.4, 0.5, 0.6])
    human_weight = 5.0 #np.random.choice([4.0, 5.0, 6.0])
    win_size = np.random.choice([10,15,20])
    if win_size == 10 :
        lpsize = [8,9,10,11,12]
        hpsize = [88,89,90,91,92]
        ransize = [8,9,10,11,12]
    elif win_size == 15 :
        lpsize = [13,14,15,16,17]
        hpsize = [83,84,85,86,87]
        ransize = [13,14,15,16,17]
    else :
        lpsize = [18,19,20,21,22]
        hpsize = [78,79,80,81,82]
        ransize = [18,19,20,21,22]
    ru = 0.5 #np.random.uniform(.1, .9)
    uv = [ru, 1-ru]
    lowpass = [uv, [7,8,9,10,11,12]] #np.random.choice([[uv ,lpsize], False])
    highpass = [uv, [80,81,82,83,84,85,86,87] ] #np.random.choice([[uv, hpsize], False])
    nm = np.random.choice([2,3,4])
    ranfilter2 = [2, [8,9,10,11,12]] #np.random.choice([[nm, ransize], False])
    tmp_string = "tmp"
    params = {'sr': sr,
          'batch_size': batch_size,
          'feature': feature,
          'n_classes': 2,
          'sec': sec,
          'filter_scale': filter_scale,
          'fmin' : fmin,
          'n_bins': int(n_bins),
          'tofile': tmp_string,
          'shuffle': True,
          'beta_param': beta_param,
          'data_dir': asv2019,
          'lowpass': lowpass,
          'highpass': highpass,
          #          'ranfilter' : [10,11,12,13,14,15]
          'ranfilter2' : ranfilter2 
          #           'dropblock' : [30, 100]
          #'device' : device
    }
    params_no_shuffle = {'sr': sr,
                     'batch_size': batch_size,
                     'feature': feature,
                     'n_classes': 2,
                     'sec': sec,
                     'filter_scale': filter_scale,
                     'fmin' : fmin,
                     'n_bins': int(n_bins),
                     'tofile': tmp_string,
                     'shuffle': False,
                    'data_dir': asv2019
                     #'device': device
    }
    training_generator = DataGenerator(dl.train, dl.labels, **params)
    validation_generator = DataGenerator(dl.dev, dl.labels, **params_no_shuffle)
    eval_generator = DataGenerator(dl.eval, dl.labels, **params_no_shuffle)
    input_shape = training_generator.get_input_shape()
    model = get_DDWSseq(input_shape)
#    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

    EPOCHS = epoch
    lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=EPOCHS))
    class_weight = {0: human_weight, 1: 1.} ## human: 0, 1: speaker
    history = model.fit_generator(generator=training_generator, #validation_data=track1_generator,
                              epochs=EPOCHS, class_weight=class_weight, callbacks=[lr], verbose=1)
    eer_val = evalEER(validation_generator, model)
    eer_eval = evalEER(eval_generator, model)
    endtxt1 = str(eer_val)[:6] + '.hdf5'
    savefnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1 )
    savefnm = 'saved_models/' + savefnm
    model.save(savefnm)
    endtxt1 = str(eer_val)[:6] + '.npy'
    fnm = gen_fname(model_name = mname, params = params , dropout_rate = str(dropout_rate), human_weight = str(human_weight), endtxt = endtxt1)
    fnm = 'saved_results/' + fnm
    sc1 = model.predict(eval_generator)
    np.save(fnm, sc1)
    params_save = params
    params_save['model'] = mname
    params_save['human_weight'] = str(human_weight)
    params_save['dropout_rate'] = str(dropout_rate)
    params_save['eer_val'] = str(eer_val)[:6]
    params_save['eer_eval'] = str(eer_eval)[:6]
    params_save['saved_model'] = savefnm
    tnow = datetime.datetime.now()
    params_save['tnow'] = str(tnow)
    print(params_save)
    fnm = 'res_blur_LA/rec'+str(tnow) + '.pickle'
    with open(fnm, 'wb') as f:
        pickle.dump(params_save, f, pickle.HIGHEST_PROTOCOL)


cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)
cqt
Input_shape:(100,282,1)


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
1586/1586 [==============================] - 77s 42ms/step - loss: 0.6089 - auc: 0.8960
Epoch 2/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3986 - auc: 0.9590
Epoch 3/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3496 - auc: 0.9649
Epoch 4/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3469 - auc: 0.9668
Epoch 5/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3337 - auc: 0.9684
Epoch 6/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3164 - auc: 0.9726
Epoch 7/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3138 - auc: 0.9718
Epoch 8/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.3179 - auc: 0.9730
Epoch 9/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.2972 - auc: 0.9746
Epoch 10/70
1586/1586 [==============================] - 38s 24ms/step - loss: 0.2946 - auc: 0.9745
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.00624713453584908


...

Epoch 1/70
1586/1586 [==============================] - 65s 38ms/step - loss: 1.3632 - auc: 0.7854
Epoch 2/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.7626 - auc: 0.9090
Epoch 3/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.6090 - auc: 0.9287
Epoch 4/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.4841 - auc: 0.9462
Epoch 5/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.4320 - auc: 0.9569
Epoch 6/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.4122 - auc: 0.9572
Epoch 7/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.4105 - auc: 0.9600
Epoch 8/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.3780 - auc: 0.9628
Epoch 9/70
1586/1586 [==============================] - 59s 37ms/step - loss: 0.4030 - auc: 0.9607
Epoch 10/70
1586/1586 [==============================] - 60s 38ms/step - loss: 0.3738 - auc: 0.9634
Epoch 11/

/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.007010013731684644


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


EER: 0.03358542088597492
{'sr': 16000, 'batch_size': 16, 'feature': 'cqt', 'n_classes': 2, 'sec': 9.0, 'filter_scale': 1, 'fmin': 5, 'n_bins': 100, 'tofile': 'tmp', 'shuffle': True, 'beta_param': 0.7, 'data_dir': '/home/ubuntu/data/asv2019/', 'lowpass': [[0.5, 0.5], [7, 8, 9, 10, 11, 12]], 'highpass': [[0.5, 0.5], [80, 81, 82, 83, 84, 85, 86, 87]], 'ranfilter2': [2, [8, 9, 10, 11, 12]], 'model': 'DDWSseq_LA_', 'human_weight': '5.0', 'dropout_rate': '0.5', 'eer_val': '0.0070', 'eer_eval': '0.0335', 'saved_model': 'saved_models/model_name!DDWSseq_LA_!dropout_rate!0.5!human_weight!5.0!feature!cqt!sec!9.0!filter_scale!1!fmin!5!n_bins!100!shuffle!True!beta_param!0.7!lowpass!--!highpass!--!ranfilter2!--!0.0070.hdf5', 'tnow': '2022-06-08 14:02:43.660051'}
